## SST plotting ##

Figuring out best way of plotting SST composites

In [ ]:
import os
import sys
import glob

import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import iris

from tqdm import tqdm

In [ ]:
# import the test file
test_file_path = "/gws/nopw/j04/canari/users/benhutch/saved_DePre/HadGEM3-GC31-MM/tos/Omon/ONDJFM/1960-1960/HadGEM3-GC31-MM_tos_ONDJFM_Omon_1960-1960.nc"

# load as a cube
cube = iris.load_cube(test_file_path)

In [ ]:
# Set up a new cube with the desired shape and metadata
new_shape = (cube.shape[0], 10, cube.shape[2], cube.shape[3])
new_data = np.zeros(new_shape, dtype=cube.data.dtype)

# Create a new cube with the same metadata as the original cube
new_cube = iris.cube.Cube(
    new_data,
    dim_coords_and_dims=[(cube.coord('time'), 0), (iris.coords.DimCoord(np.arange(1, 11), standard_name='lead', units='1'), 1), (cube.coord('latitude'), 2), (cube.coord('longitude'), 3)],
    aux_coords_and_dims=[(coord, None) for coord in cube.aux_coords if coord.name() != 'lead']
)

# Loop over winter years
for i, wyear in tqdm(enumerate(np.arange(1, 11))):
    # Set up the lead values to extract
    leads_values_this = np.arange(wyear * 12, (wyear * 12) + 6)

    # Find the indices of the lead values
    leads_indices_this = np.where(np.isin(cube.coord("lead").points, leads_values_this))[0]

    # Extract the data for the lead values
    winter_mean = cube[leads_indices_this].collapsed("lead", iris.analysis.MEAN)

    # Assign this to the new cube
    new_cube.data[:, i, :, :] = winter_mean.data

# Print the new cube to verify
print(new_cube)